In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec  2 01:41:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Parameters
num_actions_train = 2500  # Number of actions/articles in training dataset
num_actions_val = 1000    # Number of actions/articles in validation dataset
T_train = 500             # Number of time periods for outcomes per action in training
T_val = 500               # Number of time periods for outcomes per action in validation

# Data generation
np.random.seed(42)  # Set random seed for reproducibility

def mix_beta(z, x):
    Z1, Z2 = z
    if x == 1:
        alpha, beta = 25 * Z1 + 1, 25 * (1 - Z1) + 1
    else:
        alpha, beta = 25 * (1 - Z2) + 1, 25 * Z2 + 1
    return alpha, beta

# Function to generate dataset
def generate_dataset(num_actions, T):
    # Step 1: Sample Z(a) for each action (Z1, Z2) independently from Uniform(0, 0.25)
    Z = np.random.uniform(0, 0.25, size=(num_actions, 2))

    # Step 2: Initialize empty lists for X and mu_infinity
    X = np.zeros((num_actions, T))
    mu_infinity = np.zeros((num_actions, T))

    # Step 3: Generate outcomes and success rates for each time period
    Y = np.zeros((num_actions, T))
    for t in range(T):
        # Step 3.1: Sample binary feature X for each action from Bernoulli(0.5)
        X[:, t] = np.random.binomial(1, 0.5, size=num_actions)

        # Step 3.2: Sample success rate (mu_infinity) from a mixture of Beta distributions
        for i in range(num_actions):
            alpha, beta = mix_beta(Z[i], X[i, t])
            mu_infinity[i, t] = stats.beta.rvs(alpha, beta)  # Sample from the Beta distribution

        # Step 3.3: Generate outcomes Y_t for each action based on mu_infinity
        for i in range(num_actions):
            Y[i, t] = np.random.binomial(1, mu_infinity[i, t])

    return Z, X, mu_infinity, Y

# Generate training dataset
Z_train, X_train, mu_infinity_train, Y_train = generate_dataset(num_actions_train, T_train)

# Generate validation dataset
Z_val, X_val, mu_infinity_val, Y_val = generate_dataset(num_actions_val, T_val)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Convert data to PyTorch tensors
Z_train_tensor = torch.tensor(Z_train, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add new binary feature X
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
Z_val_tensor = torch.tensor(Z_val, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)  # Add new binary feature X
Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32)

# Define the Flexible Neural Network model
class FlexibleNN(nn.Module):
    def __init__(self, input_dim=23):  # Updated input_dim to include binary feature X
        super(FlexibleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, Z, X, summary_stat_0, summary_stat_1):
        # print(Z.shape, X.shape, summary_stat_0.shape, summary_stat_1.shape)
        x = torch.cat((Z, X, summary_stat_0.repeat(1, 5), summary_stat_1.repeat(1, 5)), dim=1)  # Include binary feature X
        # print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Define the Beta-Bernoulli NN model
class BetaBernoulliNN(nn.Module):
    def __init__(self, input_dim=23):  # Updated input_dim to include binary feature X
        super(BetaBernoulliNN, self).__init__()
        self.alpha_mlp = nn.Sequential(
            nn.Linear(input_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.ReLU()
        )
        self.beta_mlp = nn.Sequential(
            nn.Linear(input_dim, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.ReLU()
        )

        # Initialize bias terms to 1 to avoid starting with Beta parameters of value 0
        for layer in [self.alpha_mlp[-2], self.beta_mlp[-2]]:
            nn.init.constant_(layer.bias, 1.0)

    def forward(self, Z, X, summary_stat_0, summary_stat_1):
        x = torch.cat((Z, X, summary_stat_0.repeat(1, 5), summary_stat_1.repeat(1, 5)), dim=1)  # Include binary feature X
        alpha = self.alpha_mlp(x)
        beta = self.beta_mlp(x)
        return alpha, beta, alpha / (alpha + beta)

In [ ]:
# Training phase modification with new loss function
def train_autoregressive_model(model, train_loader, optimizer, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)  # Move model to GPU if available
    model.train()  # Set the model to training mode

    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        for Z_batch, X_batch, Y_batch in train_loader:
            # Move data to GPU if available
            Z_batch, X_batch, Y_batch = Z_batch.to(device), X_batch.to(device), Y_batch.to(device)

            # Initialize total loss for the batch
            tot_loss = 0.0

            # Sequentially calculate the loss for each time step t
            for t in range(1, Y_batch.shape[1] + 1):
                # Get the sequence of previous outcomes up to time t-1
                if t == 1:
                    previous_outcomes_0 = torch.zeros((Y_batch.shape[0], 1), device=device)  # No previous outcomes at t=1 for X=0
                    previous_outcomes_1 = torch.zeros((Y_batch.shape[0], 1), device=device)  # No previous outcomes at t=1 for X=1
                else:
                    previous_outcomes_0 = Y_batch[:, :t-1] * (1 - X_batch[:, 0, :t-1])
                    previous_outcomes_1 = Y_batch[:, :t-1] * X_batch[:, 0, :t-1]

                # Create masks to count non-zero entries
                mask_0 = (1 - X_batch[:, 0, :t-1]).sum(dim=1, keepdim=True).clamp(min=1)
                mask_1 = X_batch[:, 0, :t-1].sum(dim=1, keepdim=True).clamp(min=1)

                # Compute summary statistics for X = 0 and X = 1 separately by taking mean over non-zero entries
                stat_1_0 = previous_outcomes_0.sum(dim=1, keepdim=True) / mask_0
                stat_1_1 = previous_outcomes_1.sum(dim=1, keepdim=True) / mask_1

                stat_2_0 = torch.where((1 - X_batch[:, 0, :t-1]).sum(dim=1, keepdim=True) == 0,
                                    torch.zeros_like((1 - X_batch[:, 0, :t-1]).sum(dim=1, keepdim=True)),
                                    1 / ((1 - X_batch[:, 0, :t-1]).sum(dim=1, keepdim=True) + 1e-9))

                stat_2_1 = torch.where(X_batch[:, 0, :t-1].sum(dim=1, keepdim=True) == 0,
                                    torch.zeros_like(X_batch[:, 0, :t-1].sum(dim=1, keepdim=True)),
                                    1 / (X_batch[:, 0, :t-1].sum(dim=1, keepdim=True) + 1e-9))


                # print(stat_1_0.shape, stat_2_0.shape)
                summary_stat_t_0 = torch.cat((stat_1_0, stat_2_0), dim=1)
                summary_stat_t_1 = torch.cat((stat_1_1, stat_2_1), dim=1)
                # print(summary_stat_t_0, summary_stat_t_1)

                # print(Z_batch.shape, X_batch[:, :, t-1].shape)

                # Forward pass to get model outputs
                outputs = model(Z_batch, X_batch[:, :, t-1], summary_stat_t_0, summary_stat_t_1)

                # Compute the log-probability for the current time step t
                Y_t = Y_batch[:, t-1].unsqueeze(1)
                log_probs = Y_t * torch.log(outputs + 1e-9) + (1 - Y_t) * torch.log(1 - outputs + 1e-9)

                # Accumulate negative log-likelihood
                tot_loss += -torch.sum(log_probs)

            # Backward pass and optimization
            optimizer.zero_grad()
            tot_loss.backward()
            optimizer.step()

            running_loss += tot_loss.item()

        # Print loss for every 100 epochs
        if (epoch + 1) % 100 == 0:
            avg_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

    print("Training completed.")


In [ ]:
batch_size = 500          # Batch size for training
learning_rate = 0.001     # Learning rate for optimizer
epochs = 1000             # Number of epochs for training
weight_decay = 0.01       # Weight decay for AdamW optimizer

In [ ]:
# Initialize the model, loss function, and optimizer
model = FlexibleNN()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

train_dataset = TensorDataset(Z_train_tensor, X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

train_autoregressive_model(model, train_loader, optimizer, epochs)

 10%|█         | 100/1000 [06:41<1:00:05,  4.01s/it]

Epoch [100/1000], Loss: 101906.6438


 20%|██        | 200/1000 [13:22<53:06,  3.98s/it]

Epoch [200/1000], Loss: 101794.5734


 30%|███       | 300/1000 [19:59<46:19,  3.97s/it]

...

Epoch [800/1000], Loss: 101720.2812


 90%|█████████ | 900/1000 [59:35<06:34,  3.94s/it]

Epoch [900/1000], Loss: 101710.8516


100%|██████████| 1000/1000 [1:06:14<00:00,  3.97s/it]

Epoch [1000/1000], Loss: 101707.3141
Training completed.


In [ ]:
# Save the model state dictionary after training
model_save_path = "/content/drive/My Drive/contextual_flexible_nn_model.pth"
torch.save(model.state_dict(), model_save_path)